In [ ]:
import os
from diffsky.experimental.disk_bulge_modeling.generate_bulge_disk_sample import (
    get_bulge_disk_test_sample,
    get_bulge_disk_decomposition,
)
from diffsky.experimental.disk_bulge_modeling.disk_bulge_kernels import (
    get_observed_quantity_pop,
)

from diffaux.validation.plot_utilities import (
    get_zindexes,
)
from diffaux.black_hole_modeling.black_hole_mass import (
    bh_mass_from_bulge_mass,
    monte_carlo_black_hole_mass,
)
from diffaux.black_hole_modeling.black_hole_accretion_rate import (
    eddington_ratio_distribution,
    monte_carlo_eddington_ratio,
    monte_carlo_bh_acc_rate,
)
from jax import random as jran
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict, namedtuple
from itertools import zip_longest

In [ ]:
ran_key = jran.key(0)
halo_key, ran_key = jran.split(ran_key, 2)
lgmp_min = 11.0
redshift = 0.05
Lbox = 75.0
diffstar_cens = get_bulge_disk_test_sample(halo_key, lgmp_min=lgmp_min, redshift=redshift, Lbox=Lbox)
print(list(diffstar_cens.keys()))

In [ ]:
zvalues = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5]
redshifts = diffstar_cens["z_table"]
zindexes, zs = get_zindexes(zvalues, redshifts)

In [ ]:
disk_bulge_key, ran_key = jran.split(ran_key, 2)
diffstar_cens = get_bulge_disk_decomposition(disk_bulge_key, diffstar_cens)
print(diffstar_cens.keys())

In [ ]:
# compute bulge mass at t_obs
tarr = diffstar_cens['t_table']
print(diffstar_cens['t_obs'])
tobs = jnp.full(len(diffstar_cens['logsm_obs']), diffstar_cens['t_obs'])
print(tarr.shape, tobs.shape, diffstar_cens['smh_bulge'].shape)
bulge_sm_obs = get_observed_quantity_pop(tobs, tarr, diffstar_cens['smh_bulge'])
print(np.log10(np.min(bulge_sm_obs)), np.log10(np.max(bulge_sm_obs)))

In [ ]:
bh_mass =  bh_mass_from_bulge_mass(bulge_sm_obs)

In [ ]:
bh_key, ran_key = jran.split(ran_key, 2)
bh_mass_mc = monte_carlo_black_hole_mass(bulge_sm_obs, bh_key)
print(np.log10(np.min(bh_mass_mc)), np.log10(np.max(bh_mass_mc)))

In [ ]:
plotdir = "/Users/kovacs/cosmology/BlackHole/Plots"
nrow = 1
ncol = 2
from scipy.stats import binned_statistic
bins = np.logspace(7.5, 11.5, 32)
bulge_mean, _, _ = binned_statistic(bulge_sm_obs, bulge_sm_obs, statistic='mean', bins=bins)
bh_mean, _, _ = binned_statistic(bulge_sm_obs, bh_mass, statistic='mean', bins=bins)
bh_mean_mc, _, _ =  binned_statistic(bulge_sm_obs, bh_mass_mc, statistic='mean', bins=bins)
bh_std, _, _ =  binned_statistic(bulge_sm_obs, bh_mass_mc, statistic='std', bins=bins)

fig, ax_all = plt.subplots(nrow, ncol, figsize=(5 * ncol, 4 * nrow))
labels = ('Kormendy & Ho (2013)', 'MC realization')
for ax, bhm, label in zip(ax_all, (bh_mean, bh_mean_mc), labels):
    ax.loglog(bulge_mean, bhm, label=label)
    ax.set_xlabel('$M*_{bulge}$')
    ax.set_ylabel('$M_{BH}$')
    ax.legend(loc='best')
    if 'MC' in label:
        ax.fill_between(bulge_mean, bh_mean_mc-bh_std, bh_mean_mc+bh_std, alpha=0.4)

fn = os.path.join(plotdir, 'BH_mass_vs_bulge_mass_model.png')
plt.savefig(fn)
print('Saving {}'.format(fn))


In [ ]:
# Test Eddington
rate_table, prob_table = eddington_ratio_distribution(redshift)
print(rate_table.shape, prob_table.shape)

In [ ]:
zs = np.linspace(.05, 5.05, 11)
#print(zs)
nrow = 1
ncol = 1
fig, ax = plt.subplots(nrow, ncol, figsize=(5 * ncol, 4 * nrow))
for z in zs:
    rate_table, prob_table = eddington_ratio_distribution(z)
    ax.plot(rate_table, prob_table, label='z = {:.2f}'.format(z))
    
ax.set_xscale('log')    
ax.legend(loc='best')
fn = os.path.join(plotdir, 'eddington_ratio_distribution.png')
plt.savefig(fn)
print('Saving {}'.format(fn))    

In [ ]:
edd_key, ran_key = jran.split(ran_key, 2)
sfr_percentile = jran.uniform(edd_key, minval=1e-4, maxval=1 - 1e-4, shape=(len(bh_mass),))
eddington_ratio, accretion_rate = monte_carlo_bh_acc_rate(redshift, bh_mass, sfr_percentile)

In [ ]:
print(eddington_ratio.shape, accretion_rate.shape)

In [ ]:
mbins = np.logspace(4.0, 9.0, 20)
sbins = np.logspace(-4., 0., 20)
bh_mean, _, _ = binned_statistic(bh_mass, bh_mass, statistic='mean', bins=mbins)
sfr_mean, _, _ = binned_statistic(sfr_percentile, sfr_percentile, statistic='mean', bins=sbins)
nrow = 2
ncol = 2
fig, ax_all = plt.subplots(nrow, ncol, figsize=(5 * ncol, 4 * nrow))
ylabels = ('Accretion Rate', 'Eddington Ratio')
xlabels = ('$M*_{BH}$', 'sfr_percentile')
for ax_row, y, ylabel in zip(ax_all, (accretion_rate, eddington_ratio), ylabels):
    for ax, x, xlabel, bins in zip(ax_row, (bh_mass, sfr_percentile), xlabels, (mbins, sbins)):
        xpts, _, _ = binned_statistic(x, x, statistic='mean', bins=bins)
        ypts, _, _ =  binned_statistic(x, y, statistic='mean', bins=bins)
        ax.tick_params("y", rotation=90)
        ax.loglog(xpts, ypts, label='z = {:.2f}'.format(redshift))
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.legend(loc='best')
        #ax.set_xticks(

fn = os.path.join(plotdir, 'BH_accretion_rate_model.png')
plt.savefig(fn)
print('Saving {}'.format(fn))